In [ ]:
%pip install -q dotenv langchain_community langchain_openai pygithub langgraph pydantic flask flask_cors

# Initializing tools/functions

In [19]:
from dotenv import load_dotenv

from langchain_community.agent_toolkits.github.toolkit import GitHubToolkit
from langchain_community.utilities.github import GitHubAPIWrapper
from langchain_openai import ChatOpenAI


load_dotenv()

# Toolkit is pre-programmed api wrapper for GitHub sdk. Why does it need to be pre-programmed?
toolkit = GitHubToolkit.from_github_api_wrapper(GitHubAPIWrapper())

llm = ChatOpenAI(model="gpt-4o", temperature=0)

tools = [setattr(tool, "name", tool.mode) or tool for tool in toolkit.get_tools()]

In [7]:
for tool in tools:
    print(tool.name)

get_issues
get_issue
comment_on_issue
list_open_pull_requests
get_pull_request
list_pull_request_files
create_pull_request
list_pull_request_files
create_file
read_file
update_file
delete_file
list_files_in_main_branch
list_files_in_bot_branch
list_branches_in_repo
set_active_branch
create_branch
get_files_from_directory
search_issues_and_prs
search_code
create_review_request


## Goals
- ~~Dynamic tools~~

# Create a ReAct Agent

In [8]:
from langgraph.prebuilt import create_react_agent
from pydantic import BaseModel

# Have to setup a pydantic model for structured output, otherwise have to manually explain format to the Agent
class Author(BaseModel):
    """Respond to the user in this format."""
    name: str
    issue_count: int
    comment_count: int

# ReAct agent, a hybrid deliberative and reactive agent
agent_executor = create_react_agent(llm, tools, response_format=Author)

## Goals
- ~~Dynamic tools~~
- ~~Unstructured data~~













<div align="center">
  <h1 style="color: orange;">ReAct</h1>
</div>

- <span style="font-size: 20px;"><b>ReAct: Synergizing Reasoning And Acting In Language Models</b></span>  
  [https://arxiv.org/pdf/2210.03629](https://arxiv.org/pdf/2210.03629)
- <span style="font-size: 20px;"><b>Reason - Act - Observe - Iterate loop</b></span>
- <span style="font-size: 20px;"><b>Function/Tool calling in OpenAI/Anthropic</b></span>
- <span style="font-size: 20px;"><b>Limitations of LangGraph implementation</b></span>
  - <span style="font-size: 20px;">Only two nodes</span>
  - <span style="font-size: 20px;">No human in the loop</span>

<img src="./graph.png" alt="ReAct Flowchart" style="float: right; width: 300px;">


# Using the agent

In [16]:
import json
from flask import Flask
from flask_cors import CORS

app = Flask(__name__)
CORS(app, resources={r"/*": {"origins": "http://localhost:3000"}})

@app.route('/data', methods=['GET'])
def get_data():
    # Very limiting, has to conform to the model and available tools. Underwhelming experience.
    # But prompt looks like natural language. Very reassuring
    example_query = ("For the most prolific author of github issues, give me the number of issues they have created. "
                     "Given the author's issues, I want to know the number of comments that same author has made." )

    events = agent_executor.stream(
        {"messages": [("user", example_query)]},
        stream_mode="values",
    )
    event = {}
    for event in events:
        event["messages"][-1].pretty_print()

    # Structured output is a nice breakthrough, previously getting JSON output was not consistent
    structured_response = event.get("structured_response")
    data = {"data": [structured_response.model_dump()]}
    print(f"data: {data}", flush=True)
    return json.dumps(data)
    

## Goals
- ~~End user will write only natural language~~
- ~~Unstructured data~~
- ~~Implicit algorithms~~
- ~~Dynamic tools~~
- ~~Human in the loop built-in~~
- ~~Consistent output~~
- ~~Iterative development~~



In [17]:
def start_server():
    app.run(debug=False, port=5001, use_reloader=False)

from threading import Thread

flask_thread = Thread(target=start_server, daemon=True)
flask_thread.start()

In [11]:
!curl http://127.0.0.1:5001/data

{"data": [{"name": "yasonk", "issue_count": 2, "comment_count": 3}]}

In [15]:
flask_thread.join(timeout=1)